In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/Colab\ Notebooks/Minor\ Project\ Sem\ 6/

/content/drive/MyDrive/Colab Notebooks/Minor Project Sem 6


# Import packages

In [ ]:
import pandas as pd
import numpy as np

from ast import literal_eval # to convert array string to array
from IPython.display import clear_output # to clear the large outputs

In [ ]:
# !pip install tensorflow
# !pip install tensorflow-hub
!pip install sentence-transformers
clear_output()

In [ ]:
from sentence_transformers import SentenceTransformer
import tensorflow as tf

# Load features csv and articles data

In [ ]:
articles_df = pd.read_csv('duc2002finaldataset_0.csv')
articles_df = articles_df.iloc[:-1, :]
articles = articles_df.drop('Summary', axis=1)
articles_df.head()

,Article,Summary
0,"['On the day of the Big Event, Ladbroke, the l...","['Penelope Lively won the 1987 Booker Prize.',..."
1,"[""Australian novelist Peter Carey was awarded ...","[""The coveted Booker Prize for the year's best..."
2,"[""Six novels have been nominated for the Booke...","[""The winner of the 1989 Booker Prize, Britain..."
3,"[""Japanese writer Kazuo Ishiguro won the 1989 ...",['It was announced Thursday that Kazuo Ishigur...
4,"[""The Booker Prize is Britain's literary event...",['The Booker Prize has become internationally ...


In [ ]:
features_df = pd.read_csv('features_for_all_articles.csv')
features_df.head()

,File Number,Sentence Number,Sentence length,Sentence Position,Numeric Data,Named Entity,Special Charecters,Upper Case,Entropy,Incorrect Word,POS Tags,Term Weight,Cosine Similarity,Busy Path,Text Rank
0,F0,S0,0.20,1.00,0.00,0.10,0.06,0.0,0.29,0.09,0.18,0.59,0.79,25,0.795211
1,F0,S1,0.28,0.98,0.35,0.28,0.21,0.0,0.26,0.27,0.29,0.68,0.47,42,1.264775
2,F0,S2,0.18,0.96,0.13,0.14,0.06,0.0,0.21,0.09,0.13,0.58,0.49,41,1.238773
3,F0,S3,0.23,0.94,0.00,0.00,0.05,0.0,0.24,0.00,0.18,0.65,0.60,33,1.010622
4,F0,S4,0.20,0.92,0.06,0.07,0.02,0.0,0.21,0.00,0.16,0.62,0.60,30,0.930889


## Calculate similarity of each sentence with its corresponding article summary (Target value) 

using bert

- Embeddings from BERT model
- POOLING
- Cosine Similarity of embedding of sentances
- Value from -1 to 1

In [ ]:
sbert_model =SentenceTransformer('bert-base-nli-mean-tokens')
clear_output()

def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

def find_similarity(sentence1, sentence2):
    sentence_embeddings = sbert_model.encode([sentence1, sentence2])
    value = cosine(sentence_embeddings[0], sentence_embeddings[1])
    return round(value, 2)

# Merge and add similarity with summary in CSV file

In [9]:
all_similarity_score = []

for i in range(len(articles['Article'])):
  print(i)
  article = literal_eval(articles_df['Article'][i])
  summary = literal_eval(articles_df['Summary'][i])
  summary = "".join(summary)

  for sent in article:
    all_similarity_score.append(find_similarity(sent, summary))
  if i%20 == 0:
    clear_output()

len(all_similarity_score)

561
562
563
564
565
566


15672

In [10]:
final_features_df = features_df.iloc[:len(all_similarity_score), :].copy()

In [11]:
final_features_df['Similarity with Summary'] = all_similarity_score
final_features_df.head()

,File Number,Sentence Number,Sentence length,Sentence Position,Numeric Data,Named Entity,Special Charecters,Upper Case,Entropy,Incorrect Word,POS Tags,Term Weight,Cosine Similarity,Busy Path,Text Rank,Similarity with Summary
0,F0,S0,0.20,1.00,0.00,0.10,0.06,0.0,0.29,0.09,0.18,0.59,0.79,25,0.795211,0.62
1,F0,S1,0.28,0.98,0.35,0.28,0.21,0.0,0.26,0.27,0.29,0.68,0.47,42,1.264775,0.56
2,F0,S2,0.18,0.96,0.13,0.14,0.06,0.0,0.21,0.09,0.13,0.58,0.49,41,1.238773,0.50
3,F0,S3,0.23,0.94,0.00,0.00,0.05,0.0,0.24,0.00,0.18,0.65,0.60,33,1.010622,0.31
4,F0,S4,0.20,0.92,0.06,0.07,0.02,0.0,0.21,0.00,0.16,0.62,0.60,30,0.930889,0.63


In [12]:
final_features_df.to_csv('features_with_summary_similarity.csv', index=False)